In [1]:
# SVM
# Support vector machines (SVMs) are a set of supervised learning methods used for 
# classification, regression and outliers detection.

# The advantages of support vector machines are:
    # Effective in high dimensional spaces.
    # Still effective in cases where number of dimensions is greater than the number of samples.
    # Uses a subset of training points in the decision function (called support vectors), 
    # so it is also memory efficient.
    # Versatile: different Kernel functions can be specified for the decision function. 
    # Common kernels are provided, but it is also possible to specify custom kernels.

# The disadvantages of support vector machines include:
    # If the number of features is much greater than the number of samples, avoid over-fitting 
    # in choosing Kernel functions and regularization term is crucial.
    # SVMs do not directly provide probability estimates, these are calculated using an 
    # expensive five-fold cross-validation (see Scores and probabilities, below).

# class sklearn.svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', 
# coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, 
# class_weight=None, verbose=False, max_iter=-1, 
# decision_function_shape='ovr', break_ties=False, random_state=None)

from sklearn import svm
import pandas as pd
import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

##############################  TRAINING  ##############################################
# data
fruits = pd.read_csv('NHANES_data_stroke_train.csv')

#under sample the non-stroke
MI_positive = fruits[fruits['stroke'] == 1]
MI_negitive = fruits[fruits['stroke'] == 2].sample(frac=.03411675511751327)
fruits = pd.concat([MI_positive, MI_negitive])

#
fruit_featureNames = ["Income","Sex","Age","Race","Edu","Diastolic","Systolic","Pulse","BMI","HDL","Trig","LDL","TCHOL","kidneys_eGFR","Diabetes","CurrentSmoker","isActive","isInsured"]
X = fruits[["Income","Sex","Age","Race","Edu","Diastolic","Systolic","Pulse","BMI","HDL","Trig","LDL","TCHOL","kidneys_eGFR","Diabetes","CurrentSmoker","isActive","isInsured"]]
y = fruits['stroke']

# Define the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('scaler', StandardScaler())  # Scale the features
])
X = preprocessing_pipeline.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.50, random_state=42)

# kernals could be: ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’.              
clf = svm.SVC(kernel="linear", C=1000, probability=True)

clf.fit(X_train, y_train)
print("accuracy train : ", clf.score(X_train, y_train))
print("accuracy test : ", clf.score(X_test, y_test))

print("predicted probabilities:\n", clf.predict_proba(X_test))

##############################  PREDICIOTN  ##############################################
# load data set
new_data = pd.read_csv("DEMO.csv")

# No stroke column so get rid of it
new_data = new_data.drop(columns=['stroke'])

# get attributes
X_new = new_data[fruit_featureNames]

# imputer
X_new = preprocessing_pipeline.fit_transform(X_new)

# Make predictions on the new data, run model
new_probabilities = clf.predict_proba(X_new)[:, 0]  # for output
new_predictions = clf.predict(X_new) # unsed, just for testing ratio of MI/noMI

# Get each sample's ID and write probabilities to the output CSV
new_participant_ids = new_data['SEQN']
new_output_df = pd.DataFrame({'SEQN': new_participant_ids, 'Pred_Probability': new_probabilities})
new_output_df.to_csv('SVMpred.csv', index=False)


accuracy train :  0.725925925925926
accuracy test :  0.6888888888888889
predicted probabilities:
 [[0.558 0.442]
 [0.291 0.709]
 [0.310 0.690]
 [0.447 0.553]
 [0.358 0.642]
 [0.486 0.514]
 [0.523 0.477]
 [0.463 0.537]
 [0.399 0.601]
 [0.458 0.542]
 [0.384 0.616]
 [0.309 0.691]
 [0.345 0.655]
 [0.665 0.335]
 [0.283 0.717]
 [0.347 0.653]
 [0.306 0.694]
 [0.314 0.686]
 [0.416 0.584]
 [0.513 0.487]
 [0.482 0.518]
 [0.625 0.375]
 [0.669 0.331]
 [0.506 0.494]
 [0.531 0.469]
 [0.549 0.451]
 [0.619 0.381]
 [0.469 0.531]
 [0.639 0.361]
 [0.522 0.478]
 [0.508 0.492]
 [0.516 0.484]
 [0.286 0.714]
 [0.331 0.669]
 [0.390 0.610]
 [0.469 0.531]
 [0.536 0.464]
 [0.520 0.480]
 [0.457 0.543]
 [0.407 0.593]
 [0.699 0.301]
 [0.540 0.460]
 [0.774 0.226]
 [0.429 0.571]
 [0.698 0.302]
 [0.540 0.460]
 [0.427 0.573]
 [0.282 0.718]
 [0.596 0.404]
 [0.279 0.721]
 [0.307 0.693]
 [0.265 0.735]
 [0.568 0.432]
 [0.465 0.535]
 [0.632 0.368]
 [0.342 0.658]
 [0.438 0.562]
 [0.355 0.645]
 [0.297 0.703]
 [0.405 0.595]
 [

In [4]:
#### See which data points are critical #####
# get the support vectors
print("clf support vectors: {}".format(clf.support_vectors_))
# get indices of support vectors
print("clf support vector indices: {}".format(clf.support_))
# get number of support vectors for each class
print("clf # of support vectors in each class: {}".format(clf.n_support_))

clf support vectors: [[-1.125 -1.015 1.225 ... 0.557 0.540 -0.348]
 [0.000 0.985 -0.495 ... -1.794 0.540 -0.348]
 [-0.943 0.985 -1.068 ... 0.557 0.540 -0.348]
 ...
 [0.074 0.985 0.733 ... 0.557 0.540 -0.348]
 [-0.155 0.985 -0.086 ... -1.794 0.540 -0.348]
 [-0.903 -1.015 0.570 ... 0.557 0.540 2.882]]
clf support vector indices: [  0  13  14  15  19  22  24  32  34  37  39  40  43  46  52  53  57  61
  62  63  64  67  74  81  82  85  88  89  93  97  99 100 110 112 113 115
 123 125 132 133 134   2   3   5   6  12  16  17  31  38  41  42  45  48
  51  58  59  60  66  73  75  76  77  87  92  94 103 106 108 109 118 119
 121 122 124 127 129]
clf # of support vectors in each class: [41 36]
